A copy file to check the 91663 song

In [1]:
import pandas as pd 
import numpy as np
import os
import json 
import tiktoken
import json

import time
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv() 
API_KEY = os.environ["OPENAI_API_KEY"]



In [2]:
#4 Check the satus 
def check_openai_batch(batch_id, api_key=None):
    from openai import OpenAI

    if api_key:
        client = OpenAI(api_key=api_key)
    else:
        client = OpenAI()

    batch = client.batches.retrieve(batch_id)
    # print(batch)
    return batch
#5 Retreive or doload the results
def download_results(batch_id, api_key=None):
    client = OpenAI(api_key=api_key)
    file_response = client.files.content(batch_id)
    # print(file_response.text)
    return file_response.text


def save_embeddings_only(file_response_text,  start=0, end=0, output_path="../../../../data/gpt_responses"):
    os.makedirs(output_path, exist_ok=True)
    output_file = f"{output_path}/{start}_{end}_embeddings_only.jsonl"

    with open(output_file, "w", encoding="utf-8") as f:
        for line in file_response_text.splitlines():
            if not line.strip():
                continue
            try:
                j = json.loads(line)
                custom_id = j["custom_id"]
                embedding = j["response"]["body"]["data"][0]["embedding"]
                # Guardar como JSON en una sola línea
                f.write(json.dumps({"custom_id": custom_id, "embedding": embedding}, ensure_ascii=False) + "\n")
            except Exception as e:
                print(f"Error procesando línea: {e}")

    print(f"Embeddings guardados solo con custom_id en: {output_file}")
    return output_file

def get_first_number(filename):
    match = re.search(r"(\d+)", filename)
    return int(match.group(1)) if match else float('inf')

def get_second_number(filename: str) -> int:
    matches = re.findall(r"(\d+)", filename)
    if len(matches) >= 2:
        return int(matches[1])  # el segundo número
    return float('inf')  # si no lo encuentra

def get_data(filepath: str) -> str:
    """Abre un archivo JSON y devuelve el batch_id si existe"""
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [ ]:
# gpt_responses_path = "../../../../data/new_gpt/metadata_butches_files"
# data = get_data(os.path.join(gpt_responses_path, "batch_metadata_96814_1068124.json"))
# status = check_openai_batch(data.get("batch_id"), API_KEY)
# if status.status == "completed":
#     print("Completed")
# else : 
#     print(status.status)
#     print("Aun sigue esperando we")

Completed


In [3]:
# Datapath of the GPT responses
# gpt_responses_path = "../../../../data/new_gpt/metadata_butches_files"
gpt_responses_path = "../../../../data/new_gpt_np/metadata_butches_files"
files = os.listdir(gpt_responses_path)
files_sorted = sorted(files, key=get_first_number)
# print(files)
print(files_sorted)
all_completed = True
# Save the result with custom id and embbeding
for file_embb in files_sorted:
    print(file_embb)
    start = get_first_number(file_embb)
    end = get_second_number(file_embb)
    # Verficar end
    data = get_data(os.path.join(gpt_responses_path, file_embb))
    status = check_openai_batch(data.get("batch_id"), API_KEY)
    if status.status == "completed":
        print(f"{file_embb} .... Completed")
    else:
        all_completed = False
        break
if all_completed:
    print("Ready para descargar")

['batch_metadata_0_91663.json']
batch_metadata_0_91663.json
batch_metadata_0_91663.json .... Completed
Ready para descargar


In [4]:
if all_completed:
    print("All butches are completed, let's dowload with id to sorted them")
    # Datapath of the GPT responses
    gpt_responses_path = "../../../../data/new_gpt_np/metadata_butches_files"
    path_de_descarga = "../../../../data/new_gpt_np/embeddings_with_id"
    os.makedirs(path_de_descarga, exist_ok=True)
    files = os.listdir(gpt_responses_path)
    files_sorted = sorted(files, key=get_first_number)
    # print(files)
    print(files_sorted)

    # Save the result with custom id and embbeding
    for file_embb in files_sorted:
        print(file_embb)
        start = get_first_number(file_embb)
        end = get_second_number(file_embb)
        # Verficar end
        data = get_data(os.path.join(gpt_responses_path, file_embb))
        status = check_openai_batch(data.get("batch_id"), API_KEY)
        if status.status == "completed":
            print("Completed")
            output_file_id = status.output_file_id
            pta = download_results(output_file_id, API_KEY)
            save_embeddings_only(pta, start=start, end=end, output_path=path_de_descarga)

All butches are completed, let's dowload with id to sorted them
['batch_metadata_0_91663.json']
batch_metadata_0_91663.json
Completed
Embeddings guardados solo con custom_id en: ../../../../data/new_gpt_np/embeddings_with_id/0_91663_embeddings_only.jsonl
